# Pymol In GCP
## Setup
We need to launch the pymol server. This is done by running a subprocess on the instance which the user is able to access. This can be done by running the <b>start-gui-server.ipynb notebook</b>. Also, there is a video tutorial going through the process called GUI-Startup-example.mp4 in the nigms shared google drive under workshops called workshop 6 GUIs.
## Pymol commands
Unfortunately you can't run single commands reliably like how you can run commands in the terminal Ex. ! pwd. However, since the pymol gui has access to the files you can write .pml files and run them within PyMOL. 

### Steps:


In [1]:
# Write the pml file
with open("script.pml","w") as scriptout:
    scriptout.write("fetch 3bcq\n") # Loads protein
    scriptout.write("set seq_view,1\n") # colors protein
    scriptout.write("util.color_chains()\n") # colors protein

1a.  Select file run script and then the pml script within pymol 

1.b Alternatively, you can set your directory into the folder and then run:
pymol> @script.pml